# --- Day 3: Gear Ratios ---
You and the Elf eventually reach a gondola lift station; he says the gondola lift will take you up to the water source, but this is as far as he can bring you. You go inside.

It doesn't take long to find the gondolas, but there seems to be a problem: they're not moving.

"Aaah!"

You turn around to see a slightly-greasy Elf with a wrench and a look of surprise. "Sorry, I wasn't expecting anyone! The gondola lift isn't working right now; it'll still be a while before I can fix it." You offer to help.

The engineer explains that an engine part seems to be missing from the engine, but nobody can figure out which one. If you can add up all the part numbers in the engine schematic, it should be easy to work out which part is missing.

The engine schematic (your puzzle input) consists of a visual representation of the engine. There are lots of numbers and symbols you don't really understand, but apparently any number adjacent to a symbol, even diagonally, is a "part number" and should be included in your sum. (Periods (.) do not count as a symbol.)

Here is an example engine schematic:

    467..114..
    ...*......
    ..35..633.
    ......#...
    617*......
    .....+.58.
    ..592.....
    ......755.
    ...$.*....
    .664.598..
    
In this schematic, two numbers are not part numbers because they are not adjacent to a symbol: 114 (top right) and 58 (middle right). Every other number is adjacent to a symbol and so is a part number; their sum is 4361.

Of course, the actual engine schematic is much larger. What is the sum of all of the part numbers in the engine schematic?

In [1]:
import numpy as np
import pandas as pd
from aocd import get_data

data_raw = get_data(day=3, year=2023)
with open("input.txt", "w") as f:
    f.write(data_raw)
input = data_raw.split("\n")

## Part 1

In [2]:
test_input = [
    "467..114..",
    "...*......",
    "..35..633.",
    "......#...",
    "617*......",
    ".....+.58.",
    "..592.....",
    "......755.",
    "...$.*....",
    ".664.598..",
]
test_input

['467..114..',
 '...*......',
 '..35..633.',
 '......#...',
 '617*......',
 '.....+.58.',
 '..592.....',
 '......755.',
 '...$.*....',
 '.664.598..']

In [3]:
def solution(input):
    input_np = np.array([list(row) for row in input])
    digit_mask = np.array(
        [[ele.isdigit() and ele != "." for ele in row] for row in input_np]
    )
    # Get the indices where the digit mask is true
    digit_where = np.where(digit_mask)
    digit_indices = list(zip(digit_where[0], digit_where[1]))
    symbol_mask = np.array(
        [[not ele.isdigit() and ele != "." for ele in row] for row in input_np]
    )
    # Get the indices where the symbol mask is true
    symbol_where = np.where(symbol_mask)
    symbol_indices = list(zip(symbol_where[0], symbol_where[1]))
    # Calculate euclidean distance between the digit and symbol indices.
    distances = np.array(
        [
            [
                np.linalg.norm(np.array(digit) - np.array(symbol))
                for digit in digit_indices
            ]
            for symbol in symbol_indices
        ]
    )
    # Check if any of the distances are less than sqrt(2) (i.e. the digits touch the symbol)
    digit_touches = list(zip(digit_indices, (distances <= np.sqrt(2)).any(axis=0)))
    touches_symbol = np.zeros_like(input_np, dtype=bool)
    for digit_idx, touches in digit_touches:
        if touches:
            touches_symbol[digit_idx[0], digit_idx[1]] = True

    # Now we need to group the numbers together and sum them if any of their digits touch a symbol
    numbers = []
    for i, line in enumerate(input_np):
        tmp_num = []
        tmp_num_touches = []
        recording = False
        for j, ele in enumerate(line):
            if ele.isdigit():
                recording = True
                tmp_num.append(ele)
                tmp_num_touches.append(touches_symbol[i, j])
                if j == len(line) - 1:
                    # We have reached the end of the line
                    # Add the number to the list of numbers if it has any touches
                    if np.array(tmp_num_touches).any():
                        numbers.append(int("".join(tmp_num)))
                    tmp_num = []
                    tmp_num_touches = []
                    recording = False
            else:
                if recording:
                    # We have reached the end of the number
                    # Add the number to the list of numbers if it has any touches
                    if np.array(tmp_num_touches).any():
                        numbers.append(int("".join(tmp_num)))
                    # Reset the temporary number and touches
                    tmp_num = []
                    tmp_num_touches = []
                    recording = False
    return sum(numbers)

In [4]:
solution(test_input)

4361

In [5]:
# solution(input)

## Part 2

The engineer finds the missing part and installs it in the engine! As the engine springs to life, you jump in the closest gondola, finally ready to ascend to the water source.

You don't seem to be going very fast, though. Maybe something is still wrong? Fortunately, the gondola has a phone labeled "help", so you pick it up and the engineer answers.

Before you can explain the situation, she suggests that you look out the window. There stands the engineer, holding a phone in one hand and waving with the other. You're going so slowly that you haven't even left the station. You exit the gondola.

The missing part wasn't the only issue - one of the gears in the engine is wrong. A gear is any * symbol that is adjacent to exactly two part numbers. Its gear ratio is the result of multiplying those two numbers together.

This time, you need to find the gear ratio of every gear and add them all up so that the engineer can figure out which gear needs to be replaced.

Consider the same engine schematic again:

    467..114..
    ...*......
    ..35..633.
    ......#...
    617*......
    .....+.58.
    ..592.....
    ......755.
    ...$.*....
    .664.598..

In this schematic, there are two gears. The first is in the top left; it has part numbers 467 and 35, so its gear ratio is 16345. The second gear is in the lower right; its gear ratio is 451490. (The * adjacent to 617 is not a gear because it is only adjacent to one part number.) Adding up all of the gear ratios produces 467835.

What is the sum of all of the gear ratios in your engine schematic?

In [25]:
def solution2(input):
    input_np = np.array([list(row) for row in input])
    symbol_mask = np.array([[ele == "*" for ele in row] for row in input_np])
    # Get the indices where the symbol mask is true
    symbol_where = np.where(symbol_mask)
    symbol_indices = list(zip(symbol_where[0], symbol_where[1]))

    numbers = []
    for i, line in enumerate(input_np):
        tmp_num = []
        tmp_num_positions = []
        recording = False
        for j, ele in enumerate(line):
            end = False
            if ele.isdigit():
                recording = True
                tmp_num.append(ele)
                tmp_num_positions.append((i, j))
                if j == len(line) - 1:
                    # Reached the end of the line and thus the number
                    end = True
            else:
                if recording:
                    end = True
            if end:
                # Reached the end of the number
                # Get list of positions that surround the number
                surrounding_positions = []
                for pos in tmp_num_positions:
                    for dk in [-1, 0, 1]:
                        for dl in [-1, 0, 1]:
                            # if di == 0 and dj == 0:
                            #     # Skip the current position of the symbol
                            #     continue
                            # else:
                            pos_new = (pos[0] + dk, pos[1] + dl)
                            if (
                                (pos_new not in tmp_num_positions)
                                and (pos_new[0] >= 0)
                                and (pos_new[1] >= 0)
                            ):
                                surrounding_positions.append(pos_new)

                # Add the number to the list of numbers
                numbers.append(
                    (int("".join(tmp_num)), list(set(surrounding_positions)))
                )

                # Reset the temporary number and touches
                tmp_num = []
                tmp_num_positions = []
                recording = False

    total = 0

    for sym_pos in symbol_indices:
        nums_that_touch = []
        for num, num_pos in numbers:
            if sym_pos in num_pos:
                nums_that_touch.append(num)
        if len(nums_that_touch) == 2:
            total += nums_that_touch[0] * nums_that_touch[1]
    return total

In [26]:
solution2(test_input)

467835

In [27]:
solution2(input)

84883664